In [ ]:
# Supressing annoying warnings (!must be done first!)
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np

# Logging
from tqdm import tqdm as tqdm_text
from tqdm.notebook import tqdm as tqdm_notebook

import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polysaccharide2 as ps2
from polysaccharide2.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

# Playing with ratios

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, ClassVar, TypeVar
from math import gcd
from numbers import Number


N = TypeVar('N')
def sgnmag(num : N) -> tuple[bool, N]:
    '''Returns the sign and magnitude of a numeric-like value'''
    return num < 0, abs(num)


@dataclass(repr=False)
class Ratio:
    '''For representing fractional ratios between two objects'''
    num   : Any
    denom : Any

    # REPRESENTATION
    def __repr__(self) -> str:
        return f'{self.num}/{self.denom}'
    
    def to_latex(self) -> str:
        '''Return latex-compatible string which represent fraction'''
        return rf'\frac{{{self.num}}}{{{self.denom}}}'

    # RELATIONS
    @property
    def reciprocal(self) -> 'Ratio':
        '''Return the reciprocal of a ration'''
        return self.__class__(self.denom, self.num)


@dataclass(repr=False)
class Rational(Ratio):
    '''For representing ratios of integers'''
    num   : int
    denom : int

    # REDUCTION
    autoreduce : ClassVar[bool]=False
    
    def __post_init__(self) -> None:
        if self.__class__.autoreduce:
            self.reduce()

    def reduce(self) -> None:
        '''Reduce numerator and denominator by greatest common factor'''
        _gcd = gcd(self.num, self.denom)
        self.num=int(self.num / _gcd)
        self.denom=int(self.denom / _gcd)
    simplify = reduce # alias for convenience

    @property
    def reduced(self) -> 'Rational':
        '''Return reduced Rational equivalent to the current rational (does not modify in-place)'''
        new_rat = self.__class__(self.num, self.denom)
        new_rat.reduce()

        return new_rat
    simplifed = reduced # alias for convenience
    
    def as_proper(self) -> tuple[int, 'Rational']:
        '''Returns the integer and proper fractional component of a ratio'''
        integ, remain = divmod(self.num, self.denom)
        return integ, self.__class__(remain, self.denom)
    
    # ARITHMETIC
    def __add__(self, other : 'Rational') -> 'Rational':
        '''Sum of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) + (self.denom * other.num),
            denom=(self.denom * other.denom)
        )
    
    def __sub__(self, other : 'Rational') -> 'Rational':
        '''Difference of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) - (self.denom * other.num),
            denom=(self.denom * other.denom)
        )

    def __mul__(self, other : 'Rational') -> 'Rational':
        '''Product of two Rationals'''
        return self.__class__(
            num=self.num * other.num,
            denom=self.denom * other.denom
        )

    def __div__(self, other : 'Rational') -> 'Rational':
        '''Quotient of two Rationals'''
        return self.__class__(
            num=self.num * other.denom,
            denom=self.denom * other.num
        )
    
    def __pow__(self, power : float) -> 'Rational':
        '''Exponentiates a ratio'''
        return self.__class__(
            num=self.num**power,
            denom=self.denom**power
        )

In [ ]:
p = Rational(3, 6)
q = Rational(4, 12)

print(p, p.reciprocal, p.reduced, p+q)

In [ ]:
Rational.autoreduce = False

In [ ]:
import numpy as np
from numbers import Number

for val in (4, 4.0, 4+0j, np.pi, '4', [4], False, 'sgdfg'):
    print(val, type(val), isinstance(val, Number))

In [ ]:
from fractions import Fraction

# Testing topology load and solvation

## Defining water

In [ ]:
from rdkit import Chem

from polysaccharide2.topology import offref
from polysaccharide2.topology.topIO import save_molecule
from polysaccharide2.rdutils.labeling.molwise import assign_ordered_atom_map_nums


water_dir = Path('water_files')
water_dir.mkdir(exist_ok=True)

# rdwat = Chem.MolFromSmiles('O')
# assign_ordered_atom_map_nums(rdwat, in_place=True)
# offwat = Molecule.from_rdkit(rdwat)
water = Molecule.from_smiles('O')

# offwat.to_file('wat.pdb', file_format='pdb')

TIP3P_ATOM_CHARGES = { # NOTE : units deliberately omitted here (become applied to entire charge array)
    'H' :  0.417,
    'O' : -0.843
}

water.partial_charges = [TIP3P_ATOM_CHARGES[atom.symbol] for atom in water.atoms]*offunit.elementary_charge

save_molecule(water_dir / 'water_tip3p_oe.sdf' , water, offref.TKREGS['OpenEye Toolkit'])
save_molecule(water_dir / 'water_tip3p_rd.sdf', water, offref.TKREGS['The RDKit'])
WATER_PATH = copyfile(water_dir / 'water_tip3p_oe.sdf', water_dir / 'water_tip3p.sdf')

### Method 1 : from .SDF file (must be curated via importlib_resources)

In [ ]:
sup = Chem.SDMolSupplier(str(water_dir / 'water_tip3p_rd.sdf'), sanitize=True, removeHs=False)
sup = Chem.SDMolSupplier(str(water_dir / 'water_tip3p_oe.sdf'), sanitize=True, removeHs=False)
water = next(sup)

offwat = Molecule.from_rdkit(water)
display(offwat)
print(offwat.partial_charges)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

### Method 2 : from string block (can be included in .py file)

In [ ]:
from io import BytesIO

WATER_BLOCK_RD = '''\

    RDKit          2D

  3  2  0  0  0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    1.2990    0.7500    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2990    0.7500    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0
  1  3  1  0
M  END
>  <atom.dprop.PartialCharge>  (1) 
-0.83399999999999996 0.41699999999999998 0.41699999999999998 

$$$$

'''

WATER_BLOCK_OE = '''
  -OEChem-09192311062D

  3  2  0     0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.7500    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.3750   -0.6495    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0  0  0  0
  1  3  1  0  0  0  0
M  END
> <atom.dprop.PartialCharge>
-0.843000 0.417000 0.417000

$$$$
'''

with BytesIO(WATER_BLOCK_RD.encode('utf8')) as block_bytes:
	sup = Chem.ForwardSDMolSupplier(block_bytes, sanitize=True, removeHs=False)
	water2 = next(sup)

offwat2 = Molecule.from_rdkit(water2)
display(offwat2)
print(offwat2.partial_charges)

## Testing load using from_pdb

In [ ]:
from pathlib import Path

from polysaccharide2.topology import offref, topIO
from polysaccharide2.topology.topinfo import get_largest_offmol
from polysaccharide2.residues.partition import partition
from polysaccharide2.monomers.repr import MonomerGroup

pdb_dir  = Path('polymer_examples/compatible_pdbs')
mono_dir = Path('polymer_examples/monomer_generation/json_files/')

pdb_sub = 'simple_polymers'
# mol_name = 'PEO_PLGA'
# mol_name = 'paam_modified'
mol_name = 'polyvinylchloride'
# pdb_sub = 'proteins'
# mol_name = '6cww'

pdb = pdb_dir / pdb_sub / f'{mol_name}.pdb'
mono = mono_dir / f'{mol_name}.json'
assert(pdb.exists())
assert(mono.exists())

monogrp = MonomerGroup.from_file(mono)
rdmol = Chem.MolFromPDBFile(str(pdb))
offtop = Topology.from_pdb(pdb, _custom_substructures=monogrp.monomers, toolkit_registry=offref.TKREGS['The RDKit'])
was_partitioned = partition(offtop)
print(was_partitioned)

offmol = get_largest_offmol(offtop)

In [ ]:
from polysaccharide2.topology.offref import TKREGS
from polysaccharide2.topology import topIO
from polysaccharide2.residues.charging import application, calculation

offmol.assign_partial_charges(partial_charge_method='am1bccelf10', toolkit_registry=TKREGS['OpenEye Toolkit'])
res_chg = calculation.get_averaged_charges(offmol, monogrp)
offmol2 = application.apply_residue_charges(offmol, res_chg, in_place=False)

topIO.topology_to_sdf('pvc1.sdf', offmol.to_topology())
topIO.topology_to_sdf('pvc2.sdf', offmol2.to_topology())

## Solvation of Topologies

In [ ]:
from math import ceil
import numpy as np
import numpy.typing as npt

from polysaccharide2.topology.solvation import boxvectors, physprops
from openmm.unit import gram, centimeter, nanometer, mole, AVOGADRO_CONSTANT_NA
from openff.units.openmm import to_openmm as units_to_openmm


# PARAMETERS
density = 0.997 * (gram / centimeter**3)
exclusion = 1.3 * nanometer

# Sizing box vectors
water = Molecule.from_file(WATER_PATH)
mol_bbox = boxvectors.get_topology_bbox(offtop)
box_vecs = boxvectors.pad_box_vectors_uniform(mol_bbox, exclusion)
box_vol  = boxvectors.get_box_volume(box_vecs, units_as_openm=True)

# determining number of waters to place
N = physprops.num_mols_in_box(water.to_rdkit(), box_vol, density=density)
print(box_vol, N)

In [ ]:
topIO.topology_to_sdf('pvc.sdf', offtop)

In [ ]:
# PACKING
packtop = packmol.pack_box(
    [water],
    [N],
    offtop,
    # mass_density=1*offunit.gram/offunit.millilitre,
    box_vectors=box_vecs, 
    box_shape=packmol.UNIT_CUBE,
    center_solute='BRICK'
)

# Developing monomer port saturation

## Port-binding rules

In [ ]:
from rdkit import Chem
from rdkit.Chem import RWMol, BondType

from polysaccharide2 import rdutils
from polysaccharide2.rdutils import rdkdraw
from polysaccharide2.rdutils.rdtypes import RDAtom, RDBond, RDMol
from polysaccharide2.rdutils.labeling import atomwise, molwise
from polysaccharide2.rdutils.labeling.molwise import assign_ordered_atom_map_nums
from polysaccharide2.rdutils.amalgamation import _bonding, bonding, portlib

from polysaccharide2.genutils.decorators.functional import optional_in_place

In [ ]:
# some "normal" molecules for testing
H  = Chem.MolFromSmarts('[#1]-[1#0]')
OH = Chem.MolFromSmarts('[#1]-[O]-[2#0]')
METHYL = Chem.MolFromSmarts('[#6](-[2#0])(-[2#0])(-[3#0])(-[1#0])')
CARBONYL = Chem.MolFromSmarts('[#6]-[#6](=[#8])-[1#0]')
WITTIG = Chem.MolFromSmarts('[#6](=[1#0])(-[2#0])(-[#1])')

TEST_MOLS_NORMAL = (
    H,
    OH,
    METHYL,
    WITTIG,
    CARBONYL
)

# Pathological examples for debug
DOUBLE_MID = Chem.MolFromSmarts('[#6](-[1#0])(-[#1])=[#0]-[#6](-[#1])(-[#1])(-[2#0])')
NEUTRONIUM = Chem.MolFromSmarts('[#0]-[#0]')
GHOST_WATER = Chem.MolFromSmarts('[#1]-[#0]-[#1]')

TEST_MOLS_PATHO = (
    DOUBLE_MID,
    NEUTRONIUM,
    GHOST_WATER
)

# COMBINE FOR UNIVERSAL TESTING
TEST_MOLS = (
    *TEST_MOLS_NORMAL,
    *TEST_MOLS_PATHO
)

In [ ]:
rdkdraw.disable_substruct_highlights()

for mol in TEST_MOLS:
    assign_ordered_atom_map_nums(mol, in_place=True)
    print(portlib.get_num_ports(mol))
    display(mol)

In [ ]:
bonding.dissolve_bond(Chem.RWMol(CARBONYL), 1, 2, new_port_desig=3)

In [ ]:
WITTIG_DUAL = Chem.MolFromSmarts('[#6](=[1#0])(-[#7](-[#1])(-[#1]))(-[#1])')

def combine_rdmols(rdmol_1 : RDMol, rdmol_2 : RDMol) -> RDMol:
    rdmol_1, rdmol_2 = molwise.assign_contiguous_atom_map_nums(rdmol_1, rdmol_2, in_place=False) 
    combo = Chem.CombineMols(rdmol_1, rdmol_2) # combine into single Mol object to allow for bonding
    combo = Chem.RWMol(combo) # make combined Mol modifiable

    atom_ids = [
        [port.bridgehead.GetIdx() for port in ports]
            for ports in portlib.get_bondable_port_pairs_internal(portlib.get_ports(combo))
    ]
    print(atom_ids)

    return bonding.increase_bond_order(combo, *atom_ids[0], in_place=False)
# m1, m2 = molwise.assign_contiguous_atom_map_nums(METHYL, OH, in_place=False)
# m1, m2 = molwise.assign_contiguous_atom_map_nums(METHYL, CARBONYL, in_place=False)

In [ ]:
new_mol = combine_rdmols(WITTIG, WITTIG_DUAL)
new_mol_2 = bonding.increase_bond_order(new_mol, 0, 4, in_place=False)
new_mol_3 = combine_rdmols(new_mol_2, OH)

display(new_mol)
display(new_mol_2)
display(new_mol_3)

## Defining port combination and substitution rules

In [ ]:
from itertools import product as cartesian_product

def ports_are_bondable(port_1 : RDAtom, port_2 : RDAtom) -> bool:
    '''Determine if two port atoms can be combined into a bond'''
    port_info_1 = portlib_legacy.get_port_info(port_1)
    port_info_2 = portlib_legacy.get_port_info(port_2)

    return ( # to be bondable port, the pair must:
        port_info_1.port.GetIdx() != port_info_2.port.GetIdx()                        # not be the same port
        and port_info_1.bh_atom.GetIdx() != port_info_2.bh_atom.GetIdx()              # be bonded to different bridegehead atoms
        and port_info_1.desig == port_info_2.desig                                    # have compatible designations
        and port_info_1.inc_bond.GetBondType() ==  port_info_2.inc_bond.GetBondType() # have matching bond types
    )

def enumerate_fusable_ports(rdmol_1 : RDMol, rdmol_2 : RDMol, asAtoms : bool=True) -> list[tuple[int, int]]:
    '''Get all pairs of atoms between two Mols which have compatible neighboring ports
    Returns a dict with keys containing the bondable atoms and values containing the corresponding bond ports'''
    ports_dict_1 = portlib_legacy.get_mol_ports_dict(rdmol_1)
    ports_dict_2 = portlib_legacy.get_mol_ports_dict(rdmol_2)

    pairs_list = [
        (port_info_1.port, port_info_2.port)
            for mutual_desig in (ports_dict_1.keys() | ports_dict_2.keys()) # enumerate over pairs of ports which match designation
                for port_info_1, port_info_2 in cartesian_product(ports_dict_1[mutual_desig], ports_dict_2[mutual_desig]) # iterate over all pairs with matching designation
                    if ports_are_bondable(port_info_1.port, port_info_2.port)
    ]

    if not asAtoms:
        return [
            tuple(port.GetIdx() for port in port_pair)
                for port_pair in pairs_list
        ]
    return pairs_list

In [ ]:
from rdkit.Chem.rdchem import BondType, RWMol
from polysaccharide2.rdutils.rderrors import BondOrderModificationError
from polysaccharide2.rdutils.reactions.bonding_legacy import _decrease_bond_order

# UP-CONVERTING BONDS
def bond_order_increasable(rdmol : RDMol, *atom_pair_ids : list[int, int]) -> bool:
    '''Check if both atoms have a free neighboring port'''
    return all(
        portlib_legacy.has_neighbor_ports(rdmol.GetAtomWithIdx(atom_id))
            for atom_id in atom_pair_ids
    )

@optional_in_place
def fuse_ports(rwmol : RWMol, *port_map_nums : tuple[int, int]) -> None:
    '''Exchange two ports for a bond of one order higher in a modifiable RWMol'''
    if not portlib_legacy.ports_are_bondable(*ports):
        raise BondOrderModificationError
    
    # locate bridgehead atoms to bond, reduce port bond orders
    bh_atom_ids = []
    for port in ports:
        port_info = portlib_legacy.get_port_info(port)
        bh_atom_id = port_info.bh_atom.GetIdx()
        bh_atom_ids.append(bh_atom_id)

        _decrease_bond_order(rwmol, port.GetIdx(), bh_atom_id, in_place=True) # remove a bond between the port and the bridgehead
        
        port = rwmol.GetAtomWithIdx(port.GetIdx())
        print(atomwise.get_num_bonds(port))
        if atomwise.get_num_bonds(port) == 0:
            rwmol.RemoveAtom(port.GetIdx()) # delete port if decreasing bond order has now caused it to unbond completely

    # determine expected bond type after order increase (handle single-bond removal, specifically) 
    curr_bond = rwmol.GetBondBetweenAtoms(*bh_atom_ids)
    if curr_bond is None:
        new_bond_type = BondType.SINGLE # with no pre-existing bond, simply add a single bond
    else: 
        new_bond_type = BondType.values[curr_bond.GetBondTypeAsDouble() + 1] # with pre-existing bond, need to get the next order up by numeric lookup
        rwmol.RemoveBond(*bh_atom_ids) # also remove the existing bond for new bond creation

    # create new bond
    rwmol.AddBond(*bh_atom_ids, order=new_bond_type) # create new bond or specified order

In [ ]:
from polysaccharide2.rdutils.amalgamation import portlib_legacy
from polysaccharide2.rdutils.labeling import molwise

# mol = portlib.splice_port(METHYL, H)
# display(mol)
# OH     = Chem.MolFromSmarts('[#1D1+0:1]-[#8D2+0:2]-[3#0:3]')
# METHYL = Chem.MolFromSmarts('[#6D4+0:1](-[3#0:2])(-[2#0:3])(-[2#0:4])(-[1#0:5])')
OH     = Chem.MolFromSmiles('[#1+0:1]-[#8+0:2]-[3#0:3]', sanitize=False)
METHYL = Chem.MolFromSmiles('[#6+0:1](-[3#0:2])(-[2#0:3])(-[2#0:4])(-[1#0:5])', sanitize=False)
display(OH)
display(METHYL)
print('='*50)

# mol = portlib.saturate_ports(METHYL, OH)
# molwise.assign_ordered_atom_map_nums(mol, in_place=True)
# display(mol)

In [ ]:
for bond in METHYL.GetBonds():
    bond.SetProp()

In [ ]:
rdmol, sat_group = molwise.assign_contiguous_atom_map_nums(METHYL, OH, in_place=False) # VITAL that this is done first to ensure map
combo = Chem.CombineMols(rdmol, sat_group)
combo = Chem.RWMol(combo)

ports = enumerate_fusable_ports(rdmol, sat_group, asAtoms=True)
port_map_nums = [port.GetAtomMapNum() for port in ports[0]]
port_atoms = [combo.GetAtomWithIdx(i) for i in molwise.atom_ids_by_map_nums(combo, *port_map_nums)]

## Implementing monomer-spec SMARTS query assignment

In [ ]:
from polysaccharide2.rdutils.rdtypes import RDAtom, RDMol

def _get_compliant_atom_query(rdatom : RDAtom) -> Chem.QueryAtom:
    '''Generated monomer-specification-compliant atom query SMARTS string (<insert our citation eventually>) for a non-port atom'''
    assert(not portlib_legacy.is_port(atom)) # TODO : add special case for handling ports once port spec is solidified

    isotope      = rdatom.GetIsotope()
    atomic_num   = rdatom.GetAtomicNum()
    degree_num   = rdatom.GetDegree() # counts number of active bonds
    formal_chg   = rdatom.GetFormalCharge()
    atom_map_num = rdatom.GetAtomMapNum() # TODO : add check for nonzero map num   
    
    if isotope == 0:
        query_SMARTS = f'[#{atomic_num}D{degree_num}{formal_chg:+}:{atom_map_num}]'
    else:
        query_SMARTS = f'[{isotope}?#{atomic_num}D{degree_num}{formal_chg:+}:{atom_map_num}]'
    print(query_SMARTS)

    return Chem.AtomFromSmarts(query_SMARTS) # include explicit plus for positive formal charges

In [ ]:
PVC_SMARTS = "[1#0:7]-[#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])-[#6D4+0:4](-[#17D1+0:5])(-[#1D1+0:6])-[2#0:14]"
PVC = Chem.MolFromSmarts(PVC_SMARTS)
PVC

In [ ]:
import re

MONOMER_ATOM_QUERY = re.compile(
    r'\[(?P<isotope>\d?)' \
    r'#(?P<atomic_num>\d+?)' \
    r'(?P<valence>.*?)' \
    r':(?P<map_num>\d+?)\]'
)

DEGREE_CHARGE_QUERY = re.compile(
    r'D(?P<degree>\d{1})' \
    r'[+-](?P<formal_charge>\d+)'
)

In [ ]:
for match in re.finditer(MONOMER_ATOM_QUERY, PVC_SMARTS):
    groups = match.groupdict()

    if groups.get('isotope') == '':
        groups['isotope'] = '0'

    if (valence := groups.get('valence')):
        atom_info = re.search(DEGREE_CHARGE_QUERY, valence).groupdict()
        groups.update(atom_info)
    groups.pop('valence') # discard for both linkers and non-linkers
    groups = {
        key : int(val)
            for key, val in groups.items() # convert to ints from strings
    }

    print(groups)

In [ ]:
METHYL = Chem.MolFromSmarts('[#6D4+0](-[3#0])(-[2#0])(-[4#0])(-[1#0])')
display(METHYL)
print(Chem.MolToSmiles(METHYL))
print(Chem.MolToSmarts(METHYL))

In [ ]:
mol = METHYL

for atom in mol.GetAtoms():
    if not portlib_legacy.is_port(atom):
        if atom.GetSymbol() == 'O':
            atom.SetIsotope(1)
        query_atom = _get_compliant_atom_query(atom)
        atom.SetQuery(query_atom)
        # print(_get_compliant_atom_query(atom))

display(mol)
print('RDKit-generated mol query : ', Chem.MolToSmarts(mol), Chem.MolToSmiles(mol))

In [ ]:
for atom in mol.GetAtoms():
    print(atom.GetSmarts())

In [ ]:
Chem.MolFromSmiles('[#6&D4&+0:1](-[#0&2*:2])(-[#0&2*:3])(-[#0&2*:4])-[#0&1*:5]')

In [ ]:
sm = Chem.MolToSmarts(mol)
sm = sm.replace('&', '')

mol2 = Chem.MolFromSmarts(sm)
display(mol2)

In [ ]:
block = Chem.MolToMolBlock(mol)

In [ ]:
import pandas as pd
from polysaccharide2.genutils.iteration import iter_len

records = []
for atom in mol.GetAtoms():
    record = {
        'symbol' : atom.GetSymbol(),
        'map num' : atom.GetAtomMapNum(),
        'num bonds' : iter_len(atom.GetBonds()),
        'impl_valence' : atom.GetImplicitValence(),
        'expl_valence' : atom.GetExplicitValence(),
        'total_valence' : atom.GetTotalValence(),
    }
    records.append(record)
df = pd.DataFrame.from_records(records)
df

In [ ]:
s = '[#6D4+0:1](-[5#02*:3])(-[2#8D2+0:5]-[#1D1+0:4])(-[#8D2+0:7]-[#1D1+0:6])-[#8D2+0:9]-[#1D1+0:8]'
mol3 = Chem.MolFromSmarts(s)
display(mol3)
print(s)
print(Chem.MolToSmarts(mol3))

# Experimenting with SMARTS functional groups

In [ ]:
from polysaccharide2.monomers.substruct.functgroups import FN_GROUP_TABLE, FN_GROUP_ENTRIES
from polysaccharide2.monomers.substruct.functgroups.records import FnGroupSMARTSEntry

In [ ]:
FN_GROUP_TABLE.loc[FN_GROUP_TABLE['group_type'].str.contains('carbonyl')]

In [ ]:
smarts = FN_GROUP_ENTRIES[44].SMARTS
Chem.MolFromSmarts(smarts)

# Testing monomer loading

In [ ]:
from pathlib import Path 
from polysaccharide2.monomers.repr import MonomerGroup

p = Path('polymer_examples/monomer_generation/json_files/bisphenolA.json')
q = Path('polymer_examples/monomer_generation/json_files/naturalrubber.json')

mg1 = MonomerGroup.from_file(p)
mg2 = MonomerGroup.from_file(q)

In [ ]:
Chem.MolFromSmiles(mg2.monomers['naturalrubber'][0])

# Testing building

In [ ]:
from polysaccharide2.polymers import estimation, building

estimation.estimate_chain_len_linear(mg1, 10)

# Testing simulation I/O

In [ ]:
from pathlib import Path 
from openmm.unit import nanosecond

sp = ps2.openmmtools.records.SimulationParameters(100*nanosecond, 5, 'NVT')
sp.to_file(Path('test.json'))